Setting seed

In [1]:
import random

random.seed(10)

Importing tensorflow

In [2]:
import tensorflow as tf

Installing the Hugging Face Library

In [3]:
!pip install transformers

     |████████████████████████████████| 2.2MB 8.4MB/s 
     |████████████████████████████████| 3.3MB 51.1MB/s 
     |████████████████████████████████| 870kB 53.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=2d84247e7b9131ca10dc8b2a11c4d6d93e9f0652b3c24b81574f63095e1ed01c
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


Loading Reddit Dataset

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
lav_path = '/content/gdrive/MyDrive/Colab Notebooks/train-balanced-sarcasm.csv'
shalz_path = '/content/gdrive/MyDrive/Colab Notebooks/train-balanced-sarcasm.csv'

import pandas as pd
full_df = pd.read_csv(lav_path)

Mounted at /content/gdrive


Viewing first few rows of Reddit Dataset

In [5]:
full_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


Reporting the number of comments

In [6]:
print('Number of comments: {:,}\n'.format(full_df.shape[0]))

Number of comments: 1,010,826



Removing missing values

In [7]:
# remove na 
full_df = full_df[full_df['parent_comment'].notna() & full_df['comment'].notna()]

Selecting a subset of the data

In [8]:
# select the full data
df = full_df
df

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
...,...,...,...,...,...,...,...,...,...,...
1010821,1,I'm sure that Iran and N. Korea have the techn...,TwarkMain,reddit.com,2,2,0,2009-04,2009-04-25 00:47:52,"No one is calling this an engineered pathogen,..."
1010822,1,"whatever you do, don't vote green!",BCHarvey,climate,1,1,0,2009-05,2009-05-14 22:27:40,In a move typical of their recent do-nothing a...
1010823,1,Perhaps this is an atheist conspiracy to make ...,rebelcommander,atheism,1,1,0,2009-01,2009-01-11 00:22:57,Screw the Disabled--I've got to get to Church ...
1010824,1,The Slavs got their own country - it is called...,catsi,worldnews,1,1,0,2009-01,2009-01-23 21:12:49,I've always been unsettled by that. I hear a l...


Extracting the comments and the labels of the dataset as numpy ndarrays

In [9]:
import numpy as np

In [10]:
parent_comments = df.parent_comment.values
comments = df.comment.values
labels = df.label.values

Tokenization

In [11]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Loading BERT tokenizer...


# Tokenize and Split Data

## Combine parent comments and comments

In [12]:
combined = [list(i) for i in zip(parent_comments, comments)]

## Tokenize using batch encode

In [13]:
batch_encoded_dict = tokenizer.batch_encode_plus(
                      combined,
                      add_special_tokens = True,
                      max_length = 50,
                      pad_to_max_length = True,
                      return_token_type_ids=True,
                      return_attention_mask = True,
                      return_tensors = 'tf'
                     )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [14]:
labels = tf.convert_to_tensor(labels)

In [15]:
labels

<tf.Tensor: shape=(1010773,), dtype=int64, numpy=array([0, 0, 0, ..., 1, 1, 1])>

### Combine the training inputs into a tensorflow dataset

In [16]:
def example_to_features(input_ids, attention_masks, token_type_ids, y):
  return {"input_ids": input_ids,
          "token_type_ids": token_type_ids,
          "attention_mask": attention_masks}, y

dataset = tf.data.Dataset.from_tensor_slices((batch_encoded_dict['input_ids'], batch_encoded_dict['token_type_ids'], batch_encoded_dict['attention_mask'], 
                                              labels)).map(example_to_features)


### Split the tensorflow dataset into Training, Validation, Test

In [17]:
# Train on 10% of the data, Validate on 10% of the data, Test on 80% of the data
# Calculate the number of samples to include in each set
dataset_size = len(dataset)
train_size = int(0.1 * dataset_size)
val_size = int(0.1 * dataset_size)
test_size = int(0.8 * dataset_size)
dataset = dataset.shuffle(dataset_size)
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = dataset.skip(test_size)
test_dataset = dataset.take(test_size)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} test samples'.format(test_size))

101,077 training samples
101,077 validation samples
808,618 test samples


# Modeling with Various Hyperparameters

Define function that takes a EvalPrediction and outputs metrics as a dictionary (string to value)

In [18]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds_flat = np.argmax(predictions, axis=1).flatten()
    labels_flat = labels.flatten()
    acc = np.sum(preds_flat == labels_flat) / len(labels_flat)
    return {'accuracy': acc}

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.01, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.1

In [19]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.1, num_labels=2)

Build TFTrainer

In [20]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [21]:
import transformers

transformers.logging.set_verbosity_info()

In [22]:
trainer.train()

***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.64429796, 'learning_rate': 5e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.6232622, 'learning_rate': 4.7940008e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.6072424, 'learning_rate': 4.5880024e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.59854627, 'learning_rate': 4.3820037e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.59332687, 'learning_rate': 4.1760046e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.5889442, 'learning_rate': 3.9700066e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.58284205, 'learning_rate': 3.764008e-05, 'ep

Evaluate on test set

In [23]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.4877101908760388, 'eval_accuracy': 0.7621747625643055, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7621747625643055, 'eval_loss': 0.4877101908760388}

### Results

In [24]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

Get test predictions

In [25]:
test_predictions = trainer.predict(test_dataset).label_ids

***** Running Prediction *****
  Num examples in dataset = 808618
  Batch size = 64


In [ ]:
test_predictions

Get true test labels

In [26]:
test_numpy = list(test_dataset.as_numpy_iterator())
true_test_labels = [elem[1] for elem in test_numpy]

In [27]:
true_test_labels

[1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,


##### Generate Classification Report

In [ ]:
# accuracy
acc = accuracy_score(true_test_labels, test_predictions)
print('Accuracy: %.3f' % acc)

# precision
precision = precision_score(true_test_labels, test_predictions, average='binary', zero_division = 0)
print('Precision: %.3f' % precision)

# recall
recall = recall_score(true_test_labels, test_predictions, average='binary', zero_division = 0)
print('Recall: %.3f' % recall)

#f1
score = f1_score(true_test_labels, test_predictions, average='binary')
print('F-Measure: %.3f' % score)

target_names = ['not sarc', 'sarc']
print(classification_report(true_test_labels, test_predictions, target_names=target_names))


### Error Analysis

Detokenize test comments and put into list

In [28]:
detokenized_comments = []
for elem in test_numpy:
  elem_input_ids = elem[0]["input_ids"]
  elem_tokens = tokenizer.convert_ids_to_tokens(elem_input_ids)
  detokenized_comments.append(" ".join(elem_tokens))

In [29]:
detokenized_comments[:5]

["[CLS] This would totally fuck so many people ' s 401 ##k ' s and investments though ( as well as many corporations investments ) . Most peoples retirement are in oil companies so we don ##t want [SEP] Well that totally makes BP a great corporation then . [SEP]",
 '[CLS] And we actually have a pretty solid idea of the keys to getting out of and staying out of poverty : ( 1 ) finish school ; ( 2 ) [SEP] People in poverty need to just get a loan from their parents and start a business . [SEP]',
 '[CLS] De ##pen ##ds on the drugs in question . Nobody I know would care at all if it were pot . [SEP] Does anyone call pot drugs anymore , especially people who smoke it ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Done . But what do we name this one ? [SEP] High me ##lani ##n mask and low me ##lani ##n mask . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [30]:
fp = []
fn = []
tp = []
tn = []

for index, (truth, pred) in enumerate(zip(true_test_labels, test_predictions)):
  if truth == 0 and pred == 1: #predicted as sarcastic but actually not
    fp.append(index)
  elif truth == 1 and pred == 0: # predicted as not but actually are sarcastic
    fn.append(index)
  elif truth == 1 and pred == 1: # predicted as sarc and correct
    tp.append(index)
  else: # predicted as not and it's correct
    tn.append(index)

##### Examples that were predicted as sarcastic but are actually not sarcastic

In [31]:
[detokenized_comments[i] for i in fp]

['[CLS] De ##pen ##ds on the drugs in question . Nobody I know would care at all if it were pot . [SEP] Does anyone call pot drugs anymore , especially people who smoke it ? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Hell ya , fuck Michigan [SEP] Buck the Fuck ##eye ##s ! [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Special ##ly the blacks [SEP] Obama is white until he needs votes . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] AND THE ##N YOU AR ##E D ##IR ##EC ##TE ##D TO H ##AV ##E F ##UN TO ##NI ##G ##HT . [SEP] E ##VE ##R ##Y ##BO ##D ##Y WA ##NG CH ##UN ##G TO ##NI ##G 

##### Examples that were predicted as not sarcastic but are actually sarcastic

In [32]:
[detokenized_comments[i] for i in fn]

["[CLS] This would totally fuck so many people ' s 401 ##k ' s and investments though ( as well as many corporations investments ) . Most peoples retirement are in oil companies so we don ##t want [SEP] Well that totally makes BP a great corporation then . [SEP]",
 '[CLS] And we actually have a pretty solid idea of the keys to getting out of and staying out of poverty : ( 1 ) finish school ; ( 2 ) [SEP] People in poverty need to just get a loan from their parents and start a business . [SEP]',
 "[CLS] mildly in ##fu ##ria ##ting that you took the time to post this . . . get over it . . your friend texts like a 4th grade ##r . . and you have shit taste in food [SEP] Ye ##a because both restaurants aren ' t great [SEP]",
 "[CLS] i refuse to believe any amount of s ##hr ##oom ##s leaves you head any different than before except more en ##light ##ened . [SEP] Guys he ' s obviously right , * s ##hr ##oom ##s are natural * so they * * can ' t possibly [SEP]",
 "[CLS] This isn ' t just for Cl

Clear session

In [ ]:
tf.keras.backend.clear_session()

### Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 3e-5, No. of Epochs 2, Weight decay 0.01, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.1

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.1, num_labels=2)

Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=3e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.67973787, 'learning_rate': 3e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.69007206, 'learning_rate': 2.8764005e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.69356567, 'learning_rate': 2.7528014e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.6955756, 'learning_rate': 2.6292022e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.69658035, 'learning_rate': 2.5056028e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.6974441, 'learning_rate': 2.382004e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.69796246, 'learning_rate': 2.2584047e-05, 'e

Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.6934757741392956, 'eval_accuracy': 0.4997130985358132, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.4997130985358132, 'eval_loss': 0.6934757741392956}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 32, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 3, Weight decay 0, Trained on 202,154 examples, Validated on 101,077 examples, Tested on 707,541 examples, Hidden dropout prob 0.5

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 202154
  Num Epochs = 3.0
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 18954


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.71059334, 'learning_rate': 5e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.71020424, 'learning_rate': 4.8645274e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.7055702, 'learning_rate': 4.729056e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.7002507, 'learning_rate': 4.5935834e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.6882027, 'learning_rate': 4.458112e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.6776838, 'learning_rate': 4.3226402e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.6683379, 'learning_rate': 4.187168e-05, 'epoch'

Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 707541
  Num examples in used in evaluation = 707584
  Batch size = 64


{'eval_loss': 0.6010955059752623, 'eval_accuracy': 0.7261470016280752, 'epoch': 3.0, 'step': 18954}


{'eval_accuracy': 0.7261470016280752, 'eval_loss': 0.6010955059752623}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 32, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 3, Weight decay 0.01, Trained on 202,154 examples, Validated on 101,077 examples, Tested on 707,541 examples, Hidden dropout prob 0.6

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.6, num_labels=2)

Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 202154
  Num Epochs = 3.0
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 18954


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.7150724, 'learning_rate': 5e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.71227187, 'learning_rate': 4.8645274e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.7105165, 'learning_rate': 4.729056e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.7089663, 'learning_rate': 4.5935834e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.70774055, 'learning_rate': 4.458112e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.70652163, 'learning_rate': 4.3226402e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.7059105, 'learning_rate': 4.187168e-05, 'epoch

Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 707541
  Num examples in used in evaluation = 707584
  Batch size = 64


{'eval_loss': 0.6931508308554178, 'eval_accuracy': 0.5003801668777135, 'epoch': 3.0, 'step': 18954}


{'eval_accuracy': 0.5003801668777135, 'eval_loss': 0.6931508308554178}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.01, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.7

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.7, num_labels=2)

Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.7390617, 'learning_rate': 5e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.7330778, 'learning_rate': 4.7940008e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.72776324, 'learning_rate': 4.5880024e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.72474784, 'learning_rate': 4.3820037e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.72236055, 'learning_rate': 4.1760046e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.7201862, 'learning_rate': 3.9700066e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.71849, 'learning_rate': 3.764008e-05, 'epoch

Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.6931378611001188, 'eval_accuracy': 0.5001026414721013, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.5001026414721013, 'eval_loss': 0.6931378611001188}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.1, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.4

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.4, num_labels=2)

Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experi

{'loss': 0.6910516, 'learning_rate': 5e-05, 'epoch': 0.07913896802785692, 'step': 500}
Saving checkpoint for step 500 at ./results/checkpoint/ckpt-1
{'loss': 0.663863, 'learning_rate': 4.7940008e-05, 'epoch': 0.15827793605571383, 'step': 1000}
Saving checkpoint for step 1000 at ./results/checkpoint/ckpt-2
{'loss': 0.6478605, 'learning_rate': 4.5880024e-05, 'epoch': 0.23741690408357075, 'step': 1500}
Saving checkpoint for step 1500 at ./results/checkpoint/ckpt-3
{'loss': 0.63814825, 'learning_rate': 4.3820037e-05, 'epoch': 0.31655587211142766, 'step': 2000}
Saving checkpoint for step 2000 at ./results/checkpoint/ckpt-4
{'loss': 0.6295067, 'learning_rate': 4.1760046e-05, 'epoch': 0.3956948401392846, 'step': 2500}
Saving checkpoint for step 2500 at ./results/checkpoint/ckpt-5
{'loss': 0.6217334, 'learning_rate': 3.9700066e-05, 'epoch': 0.4748338081671415, 'step': 3000}
Saving checkpoint for step 3000 at ./results/checkpoint/ckpt-6
{'loss': 0.61595, 'learning_rate': 3.764008e-05, 'epoch': 

Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.5411117354446973, 'eval_accuracy': 0.7379860011871785, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7379860011871785, 'eval_loss': 0.5411117354446973}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.1, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.5

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.5, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.5,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

Checkpoint file ./results/checkpoint/ckpt-25 found and restoring from checkpoint
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 12500
  Will skip the first 6182 steps in the first epoch
***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Training took: 0:00:33.860070


Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.5451579967599921, 'eval_accuracy': 0.7368643648595172, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7368643648595172, 'eval_loss': 0.5451579967599921}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 5e-5, No. of Epochs 2, Weight decay 0.1, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.1

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.1, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=5e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

Checkpoint file ./results/checkpoint/ckpt-25 found and restoring from checkpoint
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 12500
  Will skip the first 6182 steps in the first epoch
***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Training took: 0:00:34.263115


Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.5358893868099525, 'eval_accuracy': 0.7389295607439652, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7389295607439652, 'eval_loss': 0.5358893868099525}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 4e-5, No. of Epochs 2, Weight decay 0.1, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.1

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.1, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=4e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

Checkpoint file ./results/checkpoint/ckpt-25 found and restoring from checkpoint
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 12500
  Will skip the first 6182 steps in the first epoch
***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Training took: 0:00:34.025350


Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.5347286029629996, 'eval_accuracy': 0.7390519885239414, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7390519885239414, 'eval_loss': 0.5347286029629996}

Clear session

In [ ]:
tf.keras.backend.clear_session()

## Training batch size 16, Evaluation batch size 64, Learning rate 4e-5, No. of Epochs 2, Weight decay 0.01, Trained on 101,077 examples, Validated on 101,077 examples, Tested on 808,618 examples, Hidden dropout prob 0.1

In [ ]:
from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments, BertConfig

config = BertConfig.from_pretrained('bert-base-cased', hidden_dropout_prob=0.1, num_labels=2)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.5.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



Build TFTrainer

In [ ]:
training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    do_train=True,
    do_eval=True,
    evaluation_strategy = "epoch",
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    learning_rate=4e-5,              # learning rate
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
)

with training_args.strategy.scope():
    # Load TFBertForSequenceClassification, the pretrained BERT model with a single 
    # linear classification layer on top. 
    model = TFBertForSequenceClassification.from_pretrained(
        "bert-base-cased", # Use the 12-layer BERT model, with cased vocab.
        config = config,   
    )

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics     # function that computes metrics 
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Tensorflow: setting up strategy
loading weights file https://huggingface.co/bert-base-cased/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/01800f4158e284e2447020e0124bc3f6aea3ac49848e744594f7cce8ee5ac0a4.a7137b2090d9302d722735af604b4c142ec9d1bfc31be7cbbe230aea9d5cfb76.h5
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are instantiating a Trainer but W&B is not

Train and Validate

In [ ]:
import transformers

transformers.logging.set_verbosity_info()

In [ ]:
trainer.train()

Checkpoint file ./results/checkpoint/ckpt-25 found and restoring from checkpoint
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 1
  Continuing training from global step 12500
  Will skip the first 6182 steps in the first epoch
***** Running training *****
  Num examples = 101077
  Num Epochs = 2.0
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Steps per epoch = 6318
  Total optimization steps = 12636


Training took: 0:00:34.002302


Evaluate on test set

In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples in dataset = 808618
  Num examples in used in evaluation = 808640
  Batch size = 64


{'eval_loss': 0.5341978893574396, 'eval_accuracy': 0.7390433320142461, 'epoch': 2.0, 'step': 12636}


{'eval_accuracy': 0.7390433320142461, 'eval_loss': 0.5341978893574396}

Clear session

In [ ]:
tf.keras.backend.clear_session()